## Observations and Insights 

In [98]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
mouse_metadata.head(),study_results.head()
merge_df = pd.merge(mouse_metadata,study_results, how = 'inner', on = 'Mouse ID')



# Display the data table for preview
merge_df.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1


In [119]:
# checkpoint
df = merge_df.copy()

In [120]:
# Checking the number of mice.
number_mice = df['Mouse ID'].nunique()
number_mice

249

In [121]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
dup_time_id_list = df['Mouse ID'].loc[df.duplicated("Mouse ID",keep='last') & df['Timepoint'] > 0]
dup_time_id_list

1       k403
3       k403
5       k403
7       k403
11      s185
        ... 
1880    z795
1884    z969
1886    z969
1888    z969
1890    z969
Name: Mouse ID, Length: 731, dtype: object

In [122]:
# Optional: Get all the data for the duplicate mouse ID. 
df_dup = pd.DataFrame()
df_dup = df.loc[df.duplicated("Mouse ID",keep='last') & df['Timepoint'] > 0]
df_dub = df_dup.reset_index()
df_dub.head()

,index,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,1,k403,Ramicane,Male,21,16,5,38.825898,0
1,3,k403,Ramicane,Male,21,16,15,34.223992,1
2,5,k403,Ramicane,Male,21,16,25,33.464577,1
3,7,k403,Ramicane,Male,21,16,35,26.546993,1
4,11,s185,Capomulin,Female,3,17,5,43.878496,0


In [123]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
df_dup_drop = df_dub.drop_duplicates('Mouse ID',keep = 'last')
df_dup_drop.head()

,index,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
3,7,k403,Ramicane,Male,21,16,35,26.546993,1
7,17,s185,Capomulin,Female,3,17,35,28.328531,1
11,27,x401,Capomulin,Female,16,15,35,30.206825,0
15,37,m601,Capomulin,Male,22,17,35,30.834357,1
19,47,g791,Ramicane,Male,11,16,35,30.960392,1


In [124]:
# Checking the number of mice in the clean DataFrame.
num_mice = df_dup_drop['Mouse ID'].count()
num_mice

222

## Summary Statistics

In [125]:
df = df_dup_drop.copy()

In [170]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 

group_drug = df.groupby('Drug Regimen')['Tumor Volume (mm3)']
summary_mean = group_drug.mean().round(2)
summary_med = group_drug.median().round(2)
summary_var = group_drug.var().round(2)
summary_std = group_drug.std().round(2)
summary_sem = group_drug.sem().round(2)

# Assemble the resulting series into a single summary dataframe.
summary_table = pd.DataFrame()
summary_table['Mean'] = summary_mean
summary_table['Med'] = summary_med
summary_table['Var'] = summary_var
summary_table['St Dev'] = summary_std
summary_table['St Err'] = summary_sem

summary_table

,Mean,Med,Var,St Dev,St Err
Drug Regimen,,,,,
Capomulin,38.21,38.95,26.74,5.17,1.03
Ceftamin,56.34,57.17,24.67,4.97,1.11
Infubinol,56.98,56.52,25.63,5.06,1.10
Ketapril,59.72,60.30,46.36,6.81,1.45
Naftisol,58.90,60.44,49.23,7.02,1.53
Placebo,57.20,58.64,53.92,7.34,1.50
Propriva,54.30,55.86,39.35,6.27,1.34
Ramicane,38.18,37.78,21.08,4.59,0.94
Stelasyn,58.23,57.61,49.60,7.04,1.54


In [177]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line
summary_table = group_drug.agg(['mean','median','var','std','sem']).round(2)
summary_table.columns = ['Mean', 'Med', 'Var', 'St Dev', 'St Err']
summary_table

,Mean,Med,Var,St Dev,St Err
Drug Regimen,,,,,
Capomulin,38.21,38.95,26.74,5.17,1.03
Ceftamin,56.34,57.17,24.67,4.97,1.11
Infubinol,56.98,56.52,25.63,5.06,1.10
Ketapril,59.72,60.30,46.36,6.81,1.45
Naftisol,58.90,60.44,49.23,7.02,1.53
Placebo,57.20,58.64,53.92,7.34,1.50
Propriva,54.30,55.86,39.35,6.27,1.34
Ramicane,38.18,37.78,21.08,4.59,0.94
Stelasyn,58.23,57.61,49.60,7.04,1.54


## Bar and Pie Charts

In [9]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pandas.



In [10]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pyplot.



In [11]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [14]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [15]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [16]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [17]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [18]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
